<a href="https://colab.research.google.com/github/sanvirrafsaan/Data-Science-Portfolio/blob/main/Churn-prediction/notebooks/02_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/My Drive/Data Science Portfolio/churn-prediction"

RAW_DIR = f"{PROJECT_ROOT}/data/raw"
INTERIM_DIR =f"{PROJECT_ROOT}/data/interim"
PROCESSED_DIR = f"{PROJECT_ROOT}/data/processed"
MODELS_DIR = f"{PROJECT_ROOT}/models"
FIGURES_DIR = f"{PROJECT_ROOT}/reports/figures"

import pathlib
for p in [RAW_DIR, INTERIM_DIR, PROCESSED_DIR, MODELS_DIR, FIGURES_DIR]:
  pathlib.Path(p).mkdir(parents=True, exist_ok = True)

#fastest way to distinguish "file never created" from "wrong path"
print("Interim contents:", os.listdir(INTERIM_DIR))

Mounted at /content/drive
Interim contents: ['interim_data.parquet']
